In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

In [3]:
prediction_hourly = np.load('/content/drive/MyDrive/COMP9491_ASTGCN_Model/PEMS04_Recent_K_3_4_5/Best_Epoch_Saved_Recent/Hourk4output_epoch_19_test.npz')
prediction_daily = np.load('/content/drive/MyDrive/COMP9491_ASTGCN_Model/PEMS04_Recent_K_3_4_5/Best_Epoch_Saved_Recent/04Daily_K4_output_epoch_18_test.npz')

##### Printing the Actual Target Tensor Shape

In [4]:
target_tensor = prediction_hourly['data_target_tensor']
print(target_tensor.shape)

(3394, 307, 12)


In [5]:
print(prediction_hourly['prediction'].shape)
print(prediction_daily['prediction'].shape)

(3394, 307, 12)
(3281, 307, 12)


In [6]:
prediction_hourly['prediction'][0]

array([[ 48.56552 ,  50.682594,  52.925625, ...,  68.643974,  70.905304,
         73.185196],
       [ 73.5274  ,  75.64261 ,  77.97575 , ...,  95.13289 ,  97.61012 ,
        100.12238 ],
       [ 93.9498  ,  97.600975, 101.45143 , ..., 128.1253  , 131.7355  ,
        135.25557 ],
       ...,
       [ 39.083008,  41.25925 ,  43.580215, ...,  60.41503 ,  62.810184,
         65.27415 ],
       [ 18.89903 ,  19.254303,  19.787512, ...,  23.980368,  24.822813,
         25.756733],
       [ 31.0466  ,  32.024437,  33.10582 , ...,  41.76773 ,  43.244728,
         44.732533]], dtype=float32)

In [14]:
prediction_hourly['prediction'][0].shape

(307, 12)

In [15]:
print(type(prediction_hourly['prediction']))

<class 'numpy.ndarray'>


In [16]:
prediction_hourly_np = prediction_hourly['prediction']
#prediction_hourly_target = prediction_hourly['data_target_tensor']

In [17]:
# Create the original and new indices
original_indices = np.arange(prediction_hourly_np.shape[0])  # [0, 1, 2, ..., 5640]
new_indices = np.linspace(0, prediction_hourly_np.shape[0] - 1, num=3281)

In [18]:
import numpy as np
import torch
from scipy.interpolate import interp1d

In [19]:
interpolated_prediction = np.zeros((3281, 307, 12))

In [20]:
interpolated_prediction.shape

(3281, 307, 12)

In [21]:
for i in range(prediction_hourly_np.shape[1]):  # Loop over the second dimension (883)
    for j in range(prediction_hourly_np.shape[2]):  # Loop over the third dimension (12)
        # Create the interpolation function for the 1D array prediction_hourly_np[:, i, j]
        f = interp1d(original_indices, prediction_hourly_np[:, i, j], kind='linear')

        # Interpolate values at new_indices and assign to the new tensor
        interpolated_prediction[:, i, j] = f(new_indices)


In [22]:
interpolated_prediction.shape

(3281, 307, 12)

In [23]:
prediction_daily['prediction'].shape

(3281, 307, 12)

In [24]:
interpolated_prediction[0]

array([[ 48.56552124,  50.6825943 ,  52.92562485, ...,  68.6439743 ,
         70.90530396,  73.18519592],
       [ 73.52739716,  75.64260864,  77.97575378, ...,  95.13288879,
         97.61012268, 100.12238312],
       [ 93.94979858,  97.60097504, 101.45143127, ..., 128.12530518,
        131.73550415, 135.25556946],
       ...,
       [ 39.08300781,  41.25925064,  43.58021545, ...,  60.41503143,
         62.81018448,  65.27414703],
       [ 18.89903069,  19.25430298,  19.78751183, ...,  23.98036766,
         24.82281303,  25.75673294],
       [ 31.04660034,  32.02443695,  33.1058197 , ...,  41.76773071,
         43.24472809,  44.7325325 ]])

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
import torch
import torch.nn as nn
import torch.nn.init as init

class MultiComponentFusion(nn.Module):
    def __init__(self, num_nodes, num_timesteps,in_channels, hourly_bias):
        super(MultiComponentFusion, self).__init__()

        # Initialize weight matrices for each component
        self.Wh = nn.Parameter(torch.empty(num_nodes, num_timesteps))
        self.Wd = nn.Parameter(torch.empty(num_nodes, num_timesteps))

        # Apply Xavier uniform initialization
        init.xavier_uniform_(self.Wh)
        init.xavier_uniform_(self.Wd)

        # Apply sigmoid to ensure weights are between 0 and 1
        self.sigmoid = nn.Sigmoid()
        #self.TAt = Temporal_Attention_layer(in_channels, num_nodes, num_timesteps)

        self.hourly_bias = hourly_bias

    def forward(self, y_actual, y_predicted_hourly, y_predicted_daily):
        # Convert NumPy arrays to PyTorch tensors
        y_actual = torch.tensor(y_actual).float() # Convert to tensor and cast to float
        y_predicted_hourly = torch.tensor(y_predicted_hourly).float() # Convert to tensor and cast to float
        y_predicted_daily = torch.tensor(y_predicted_daily).float() # Convert to tensor and cast to float

        # Apply sigmoid to weights
        Wh = self.sigmoid(self.Wh).detach()
        Wd = self.sigmoid(self.Wd).detach()

        # Expand weights to match input dimensions
        Wh = Wh.unsqueeze(0)  # Shape becomes (1, 883, 12)
        Wd = Wd.unsqueeze(0)

        # Perform element-wise multiplication and sum
        #Y_fused = Wh * y_predicted_hourly + Wd * y_predicted_daily
        # Cast input tensors to float32

        #Wh = Wh * self.hourly_bias
        #Wd = Wd * (1 - self.hourly_bias)




        # Perform element-wise multiplication
        Yh_weighted = Wh * y_predicted_hourly
        Yd_weighted = Wd * y_predicted_daily


        Y_fused = Yh_weighted + Yd_weighted


        #num_of_samples, num_of_nodes, num_of_timesteps = Y_fused.shape
        #Y_fused = Y_fused.unsqueeze(2)
        #print(Y_fused.shape)
        #temporal_Att = self.TAt(Y_fused)
        #Y_fused = Y_fused.squeeze(2)
        #Y_fused = torch.matmul(Y_fused, temporal_Att).reshape(self.num_of_samples, num_nodes, num_timesteps)

        return Y_fused


num_samples_test_set = 3281
num_nodes = 307
num_timesteps = 12
in_channels = 1
num_of_vertices = num_nodes
hourly_bias = 0.7

# Create an instance of the fusion model
##USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device('cuda:0')
#print("CUDA:", USE_CUDA, DEVICE)
fusion_model = MultiComponentFusion(num_nodes, num_timesteps, in_channels, hourly_bias)

# Example
Y_actual = prediction_daily['data_target_tensor']
Yh = interpolated_prediction
Yd = prediction_daily['prediction']

Y_final = fusion_model(Y_actual, Yh, Yd)

In [27]:
Y_final.shape

torch.Size([3281, 307, 12])

In [28]:
Y_final[0]

tensor([[208.2610, 200.9919, 201.8241,  ..., 217.2425, 210.5018, 226.7408],
        [152.4625, 142.8112, 147.8215,  ..., 171.6072, 162.2003, 162.0053],
        [209.1220, 206.5948, 206.6208,  ..., 223.3842, 210.7655, 220.0985],
        ...,
        [300.3957, 289.1146, 280.8647,  ..., 302.9821, 298.7837, 304.6298],
        [145.2480, 129.6155, 147.8997,  ..., 153.4240, 145.6204, 142.5607],
        [141.7390, 134.8497, 134.3932,  ..., 140.8401, 151.5921, 151.7083]])

In [29]:
import torch.optim as optim

In [30]:
from sklearn.metrics import mean_absolute_error

# Ensure tensors are detached and moved to CPU before converting to numpy
Y_final_np = Y_final.detach().cpu().numpy()
Y_actual_np = Y_actual

# Flatten the tensors to 1D arrays for MAE calculation
Y_final_flat = Y_final_np.reshape(-1, 1)
Y_actual_flat = Y_actual_np.reshape(-1, 1)

# Calculate Mean Absolute Error
mae = mean_absolute_error(Y_final_flat, Y_actual_flat)

print("Mean Absolute Error:", mae)


Mean Absolute Error: 69.22746


In [34]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Ensure tensors are detached and moved to CPU before converting to numpy
Y_final_np = Y_final.detach().cpu().numpy()
Y_actual_np = Y_actual

# Flatten the tensors to 1D arrays for RMSE calculation
Y_final_flat = Y_final_np.reshape(-1, 1)
Y_actual_flat = Y_actual_np.reshape(-1, 1)

# Calculate Mean Squared Error
mse = mean_squared_error(Y_final_flat, Y_actual_flat)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 94.93367


In [ ]:
# Assuming you have component outputs Y_h, Y_d, Y_w and true values Y
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error

# Define weights as learnable parameters
W_h = torch.nn.Parameter(torch.randn(1))
W_d = torch.nn.Parameter(torch.randn(1))

# Define optimizer
optimizer = optim.Adam([W_h, W_d], lr=0.1)

criterion = nn.MSELoss()

def RMSE_Loss(input, target):
  return torch.sqrt(criterion(input, target))

final_prediction_value = []

# Training loop
for epoch in range(100):
    # Weighted sum of component outputs
    Y_pred = fusion_model(Y_actual,Yh, Yd)

    # Ensure Y_actual is a float32 tensor
    if not isinstance(Y_actual, torch.Tensor):
        Y_actual = torch.tensor(Y_actual, dtype=torch.float32, requires_grad=True) # Convert to float32
    elif Y_actual.dtype != torch.float32:
        Y_actual = Y_actual.float() # Convert to float32 if necessary
    elif not Y_actual.requires_grad:
        Y_actual.requires_grad_(True)


    Y_pred_np = Y_pred.detach()
    # Y_actual is already a tensor, no need to detach

    # Compute loss (e.g., Mean Squared Error)
    loss = RMSE_Loss(Y_pred_np, Y_actual)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss for monitoring
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        final_prediction_value.append(Y_pred_np)

# The learned weights are in W_h, W_d, W_w

RuntimeError: Found dtype Double but expected Float